# PART 2 - CAN WE OMIT SOME CONTROLS?

# Causal structure (todas las flechas con efecto verdadero = 1):
1. X → Y
2. Z1 → X,  Z1 → Y
3. Z2 → X,  Z2 → Y
4. Z3 → Z2, Z3 → Y

In [4]:
!pip install statsmodels

In [6]:
import sys
print(sys.executable)


C:\Users\ARIANA\anaconda3\envs\espacio_py39\python.exe


In [8]:
!{sys.executable} -m pip install statsmodels



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\ARIANA\anaconda3\envs\espacio_py39\python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import statsmodels.api as sm
from scipy import stats

In [ ]:
# -----------------------------
# 2) Simulación acorde al DAG
#    Todas las flechas con coeficiente 1.
# -----------------------------
rng = np.random.default_rng(42)
n = 5_000

# Exógenas
eps_z1 = rng.normal(0, 1, n)
eps_z3 = rng.normal(0, 1, n)
eps_z2 = rng.normal(0, 1, n)
eps_x  = rng.normal(0, 1, n)
eps_y  = rng.normal(0, 1, n)

Z1 = eps_z1
Z3 = eps_z3
Z2 = Z3 + eps_z2             # Z3 → Z2
X  = Z1 + Z2 + eps_x         # Z1 → X y Z2 → X
Y  = X + Z1 + Z2 + Z3 + eps_y  # X,Z1,Z2,Z3 → Y

df = pd.DataFrame({"Y": Y, "X": X, "Z1": Z1, "Z2": Z2, "Z3": Z3})

In [ ]:
# -----------------------------
# 3) Regressions requeridas
#    Guardamos el coef. de X, su IC al 99% y significancia al 1%.
# -----------------------------
def run_ols(y, Xvars, data):
    Xmat = sm.add_constant(data[Xvars])
    model = sm.OLS(data[y], Xmat).fit()
    b = model.params["X"]
    se = model.bse["X"]
    # IC 99%
    tcrit = stats.t.ppf(1 - 0.01/2, df=len(data) - Xmat.shape[1])
    lo, hi = b - tcrit*se, b + tcrit*se
    sig1 = (lo > 0) or (hi < 0)   # excluye cero al 1%
    return b, se, lo, hi, sig1, model

specs = [
    ("(1) Y ~ X"                 , ["X"]),
    ("(2) Y ~ X + Z1"            , ["X","Z1"]),
    ("(3) Y ~ X + Z2"            , ["X","Z2"]),
    ("(4) Y ~ X + Z1 + Z2"       , ["X","Z1","Z2"]),
    ("(5) Y ~ X + Z1 + Z2 + Z3"  , ["X","Z1","Z2","Z3"]),
]

rows = []
models = {}
for name, Xvars in specs:
    b, se, lo, hi, sig1, m = run_ols("Y", Xvars, df)
    rows.append({"spec": name, "beta_X": b, "se_X": se, "lo99": lo, "hi99": hi, "sig@1%": sig1})
    models[name] = m

res = pd.DataFrame(rows)
print("Efecto verdadero de X→Y = 1.0")
print(res.to_string(index=False, float_format=lambda v: f"{v:,.3f}"))

In [ ]:
# -----------------------------
# 4) Gráfico: estimaciones puntuales e IC 99% de β_X
# -----------------------------
plt.figure(figsize=(8,5.2))
x = np.arange(len(res))
y = res["beta_X"].values
yerr = np.vstack([y - res["lo99"].values, res["hi99"].values - y])

plt.errorbar(x, y, yerr=yerr, fmt="o", capsize=5, linewidth=2)
plt.axhline(1.0, linestyle="--", linewidth=1.5, label="Valor verdadero β=1")
plt.xticks(x, res["spec"].values, rotation=15)
plt.ylabel("Estimación de β_X (IC 99%)")
plt.title("Estimaciones de β_X con diferentes controles", weight="bold")
plt.grid(True, linestyle=":", alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()


# Comentario breve (en consola)
Está sesgado porque Z1, Z2 y Z3 causan Y y además Z1 y Z2 causan X.
Agregar Z1 o Z2 reduce el sesgo parcialmente. Con Z1+Z2 el sesgo casi desaparece.
Incluir Z3 también ayuda porque Z3 mueve a Y y a Z2 (vía Z3→Z2), cerrando caminos adicionales.

In [ ]:
# Respuestas a las preguntas

## 1. ¿Qué regresiones estiman correctamente el efecto de X sobre Y?

Al observar la tabla `res` y el gráfico de intervalos de confianza, las regresiones que entregan un estimador \(\hat{\beta}_X \approx 1\) (el valor verdadero) y con intervalo de confianza al 99% que contiene 1 son:

- **(4) `Y ~ X + Z1 + Z2`**  
- **(5) `Y ~ X + Z1 + Z2 + Z3`**

**Explicación causal:**  
Z1 y Z2 son confusores porque afectan tanto a X como a Y. Al incluirlos como controles, bloqueamos los caminos de back-door. En cambio, Z3 no es confusor de X (no hay flecha Z3 → X), por lo que no es estrictamente necesario incluirlo. Por eso el modelo (4) ya identifica correctamente el efecto.

---

## 2. Tabla resumen para regresiones (4) y (5)

En el código original guardamos los modelos en `models`. Con el siguiente bloque obtenemos los resultados:

```python
# Resúmenes completos
print(models["(4) Y ~ X + Z1 + Z2"].summary())
print(models["(5) Y ~ X + Z1 + Z2 + Z3"].summary())

# Tabla compacta solo para el coeficiente de X
def tabla_x(m):
    ci = m.conf_int(alpha=0.01).loc["X"]
    return pd.Series({
        "beta_X": m.params["X"],
        "se_X": m.bse["X"],
        "t": m.tvalues["X"],
        "p": m.pvalues["X"],
        "lo99": ci[0],
        "hi99": ci[1]
    })

tab45 = pd.DataFrame({
    "(4) X": tabla_x(models["(4) Y ~ X + Z1 + Z2"]),
    "(5) X": tabla_x(models["(5) Y ~ X + Z1 + Z2 + Z3"])
}).T

print(tab45.to_string(float_format=lambda v: f"{v:,.3f}"))


**Comentario esperado:**

- Ambas regresiones entregan \(\hat{\beta}_X \approx 1\), altamente significativas al 1%.  
- La precisión mejora en (5), ya que al incluir Z3 se reduce la varianza residual de Y, lo que disminuye la desviación estándar.  
- No obstante, el modelo (4) ya es suficiente para recuperar el efecto verdadero.  

---

## 3. ¿Se puede ignorar alguna Z y aun así estimar bien?

Sí. Podemos **ignorar Z3** y aún obtener una buena estimación del efecto de X sobre Y.  

**Razones:**
- **Z1 y Z2** son confusores porque influyen tanto en X como en Y. Por lo tanto, deben incluirse en el modelo.  
- **Z3** no es confusor de X, ya que no existe la flecha Z3 → X. Su efecto sobre Y ocurre de forma directa y a través de Z2.  
- El **conjunto de ajuste mínimo** por el criterio de back-door es \(\{Z1, Z2\}\).  

Por eso, la regresión **(4) `Y ~ X + Z1 + Z2`** es suficiente para obtener un estimador correcto y no sesgado del efecto causal de X sobre Y.